**Connect google drive**

In [21]:
yolov7_file = False
yolov7_tiny_file = False

In [2]:
import os
import glob
import  shutil

from sklearn.model_selection import train_test_split

In [3]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Jan  3 09:07:12 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [5]:
%cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [6]:
!pwd

/content/gdrive/MyDrive


In [7]:
if not os.path.isdir('yolov7'):
  os.makedirs('yolov7')

In [8]:
%cd yolov7

/content/gdrive/MyDrive/yolov7


**1) Clone, configure**



In [10]:
# Download YOLOv7 repository and install requirements
if yolov7_file:
  !git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!pip install -r requirements.txt

/content/gdrive/MyDrive/yolov7/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 6.6 MB/s 


In [11]:
!pwd

/content/gdrive/MyDrive/yolov7/yolov7


**2) Create a folder and unzip image dataset**

In [12]:
if yolov7_file:
  !mkdir data/obj
  !unzip /content/gdrive/MyDrive/yolov7/dataset.zip -d data/obj

**3) Split train and test**

In [13]:
if yolov7_file:

  images_list = glob.glob("/content/gdrive/MyDrive/yolov7/yolov7/data/obj/dataset/*.jpg")

  with open("/content/gdrive/MyDrive/yolov7/yolov7/data/images.txt", "w") as f:
      f.write("\n".join(images_list))

  label_list = glob.glob("/content/gdrive/MyDrive/yolov7/yolov7/data/obj/dataset/*.txt")
  with open("/content/gdrive/MyDrive/yolov7/yolov7/data/annotations.txt", "w") as f:
      f.write("\n".join(label_list))


In [14]:
if yolov7_file:  
  
  # Read images and annotations
  #images = [os.path.join('images', x) for x in os.listdir('images')]
  #annotations = [os.path.join('annotations', x) for x in os.listdir('annotations.txt') if x[-3:] == "txt"]

  images_list.sort()
  label_list.sort()

  # Split the dataset into train-valid-test splits 
  train_images, val_images, train_annotations, val_annotations = train_test_split(images_list, label_list, test_size = 0.2, random_state = 1)
  val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)


In [15]:
%cd /content/gdrive/MyDrive/yolov7/yolov7/data
if yolov7_file:
  !mkdir train   test   val
  !mkdir train/images  train/labels   test/images  test/labels  val/images  val/labels


/content/gdrive/MyDrive/yolov7/yolov7/data


In [16]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
if yolov7_file:
  move_files_to_folder(train_images, './train/images/')
  move_files_to_folder(val_images, './val/images/')
  move_files_to_folder(test_images, './test/images/')
  move_files_to_folder(train_annotations, './train/labels/')
  move_files_to_folder(val_annotations, './val/labels/')
  move_files_to_folder(test_annotations, './test/labels/')

%cd ../

/content/gdrive/MyDrive/yolov7/yolov7


**4) Create  a custom configuration file**

In [17]:
if yolov7_file:
  !echo -e 'train : /content/gdrive/MyDrive/yolov7/yolov7/data/train\ntest: /content/gdrive/MyDrive/yolov7/yolov7/data/test\nval: /content/gdrive/MyDrive/yolov7/yolov7/data/val\n#Classes\nnc: 1\nnames: ["maiz14"]'> data/custom.yaml

In [18]:
if yolov7_file:
  !cp /content/gdrive/MyDrive/yolov7/yolov7/cfg/training/yolov7.yaml  /content/gdrive/MyDrive/yolov7/yolov7/cfg/training/yolov7_custom.yaml


In [19]:
# Change lines in yolov7.cfg file
if yolov7_file:
  !sed -i '2 s@nc: 80@nc: 1@' /content/gdrive/MyDrive/yolov7/yolov7/cfg/training/yolov7_custom.yaml

Configuration file for yolov7-tiny

In [20]:
%cd /content/gdrive/MyDrive/yolov7/yolov7

/content/gdrive/MyDrive/yolov7/yolov7


In [22]:
if yolov7_tiny_file:
  !cp /content/gdrive/MyDrive/yolov7/yolov7/cfg/training/yolov7-tiny.yaml  /content/gdrive/MyDrive/yolov7/yolov7/cfg/training/yolov7_custom-tiny.yaml

In [23]:
# Change lines in yolov7.cfg file
if yolov7_tiny_file:
  !sed -i '2 s@nc: 80@nc: 1@' /content/gdrive/MyDrive/yolov7/yolov7/cfg/training/yolov7_custom-tiny.yaml

**5) Download pre-trained weights for the convolutional layers file**

In [24]:
%cd /content/gdrive/MyDrive/yolov7/yolov7
if yolov7_file:
  !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7_training.pt

/content/gdrive/MyDrive/yolov7/yolov7


In [25]:
%cd /content/gdrive/MyDrive/yolov7/yolov7
if yolov7_tiny_file:
  !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7-tiny.pt

/content/gdrive/MyDrive/yolov7/yolov7


**6) Start training**

In [ ]:
#!python3 train.py --weights yolov7_training.pt --data "data/custom.yaml" --workers 4 --batch-size 4 --img 416 --cfg cfg/training/yolov7_custom.yaml --name yolov7 --hyp data/hyp.scratch.p5.yaml --epochs 50

In [26]:
# run this cell to begin training
%cd /content/gdrive/MyDrive/yolov7/yolov7
#!python train.py --workers 1  --batch 8 --epochs 100 --data 'data/custom.yaml'  --weights 'yolov7-tiny.pt' --device 0 --img 640 640  --hyp data/hyp.scratch.custom.yaml  --cfg cfg/training/yolov7_custom-tiny.yaml  --name yolov7-custom-tiny
!python train.py --workers 1  --batch 8 --epochs 100 --data 'data/custom.yaml'  --weights 'yolov7-tiny-best.pt' --device 0 --img 640 640  --hyp data/hyp.scratch.custom.yaml  --cfg cfg/training/yolov7_custom-tiny.yaml  --name yolov7-custom-tiny

/content/gdrive/MyDrive/yolov7/yolov7
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.0+cu116 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=8, bbox_interval=-1, bucket='', cache_images=False, cfg='cfg/training/yolov7_custom-tiny.yaml', data='data/custom.yaml', device='0', entity=None, epochs=100, evolve=False, exist_ok=False, freeze=[0], global_rank=-1, hyp='data/hyp.scratch.custom.yaml', image_weights=False, img_size=[640, 640], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='yolov7-custom-tiny', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/yolov7-custom-tiny', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=8, upload_dataset=False, v5_metric=False, weights='yolov7-tiny-best.pt', workers=1, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
hyperparameters: lr0=0.01, lrf=

**7) Detection**

In [27]:
%cd /content/gdrive/MyDrive/yolov7/yolov7

/content/gdrive/MyDrive/yolov7/yolov7


In [28]:
# Detection
!python detect.py --weights /content/gdrive/MyDrive/yolov7/yolov7/runs/train/yolov7-custom-tiny/weights/best.pt --conf 0.25 --img-size 640 --source /content/gdrive/MyDrive/yolov7/yolov7/data/test/images

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, img_size=640, iou_thres=0.45, name='exp', no_trace=False, nosave=False, project='runs/detect', save_conf=False, save_txt=False, source='/content/gdrive/MyDrive/yolov7/yolov7/data/test/images', update=False, view_img=False, weights=['/content/gdrive/MyDrive/yolov7/yolov7/runs/train/yolov7-custom-tiny/weights/best.pt'])
YOLOR 🚀 v0.1-121-g2fdc7f1 torch 1.13.0+cu116 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
IDetect.fuse
/usr/local/lib/python3.8/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 208 layers, 6007596 parameters, 0 gradients, 13.0 GFLOPS
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 

13

In [34]:
#display inference on ALL test images

import glob
from IPython.display import Image, display

i = 5
limit = i + 5 # max images to print
for imageName in glob.glob('/content/gdrive/MyDrive/yolov7/yolov7/runs/detect/exp3/*.jpg'): #assuming JPG
    if i < limit:
      display(Image(filename=imageName,width = 500, height = 430))
      print("\n")
    i = i + 1
    

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# define helper functions to show images
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

In [ ]:
imShow("runs/detect/exp2/m13-m14-0046.jpg")